# 1D Harmonic Oscillator
[Link to tutorial](https://octopus-code.org/documentation/13/tutorial/model/1d_harmonic_oscillator/)

As a first example we use the standard textbook harmonic oscillator in one dimension and fill it with two non-interacting electrons.

### Input

The first thing to do is to tell **Octopus** what we want it to do. Write the following lines and save the file as inp.

In [ ]:
import matplotlib.pyplot as plt

from postopus import Run

In [ ]:
!mkdir -p 1-1d-harmonic-oscillator

In [ ]:
cd 1-1d-harmonic-oscillator

In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

FromScratch = yes
CalculationMode = gs

Dimensions = 1
TheoryLevel = independent_particles

Radius = 10
Spacing = 0.1

%Species
  "HO" | species_user_defined | potential_formula | "0.5*x^2" | valence | 2
%

%Coordinates
  "HO" | 0
%

Most of these input variables should already be familiar. Here is a more detailed explanation for some of the values:

* [Dimensions](https://www.octopus-code.org/documentation//13/variables/system/dimensions) = 1: This means that the code should run for 1-dimensional systems. Other options are 2, or 3 (the default). You can actually run in 4D too if you have compiled with the configure flag –max-dim=4.

* [TheoryLevel](https://www.octopus-code.org/documentation//13/variables/hamiltonian/theorylevel) = independent_particles: We tell **Octopus** to treat the electrons as non-interacting.

* [Radius](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/radius) = 10.0: The radius of the 1D “sphere,” ‘‘i.e.’’ a line; therefore domain extends from -10 to +10 bohr.

* %[Species](https://www.octopus-code.org/documentation//13/variables/system/species/species): The species name is “HO”, then the potential formula is given, and finally the number of valence electrons. See [Input file](https://www.octopus-code.org/documentation//13/manual/input_file) for a description of what kind of expressions can be given for the potential formula.

* %[Coordinates](https://www.octopus-code.org/documentation//13/variables/system/coordinates/coordinates): add the external potential defined for species “HO” in the position (0,0,0). The coordinates used in the potential formula are relative to this point.

### Output

Now one can execute this file by running Octopus. 

In [ ]:
!octopus

Here are a few things worthy of a closer look in the standard output.

In [ ]:
run = Run(".")
info = run.default.scf.info()
total_energy = [lines for lines in info if "Total       = " in lines][-1]
total_energy = float(total_energy.split("=")[-1])
print(total_energy)

First one finds the listing of the species:

In [ ]:
!cat stdout_gs.txt | grep -A 5 "[*] Species [*]"

The potential is $V(x) = 0.5 x^2$, and 5 Hermite-polynomial orbitals are available for LCAO (the number is based on the valence). The theory level is as we requested:

In [ ]:
!cat stdout_gs.txt | grep -A 3 "[*] Space [*]"

The electrons are treated as “non-interacting”, which means that the Hartree and exchange-correlation terms are not included. This is usually appropriate for model systems, in particular because the standard XC approximations we use for actual electrons are not correct for “effective electrons” with a different mass.

 Input: [[MixField](https://www.octopus-code.org/documentation//13/variables/scf/mixing/mixfield) = none] (what to mix during SCF cycles)

Since we are using independent particles (and only one electron) there is no need to use a mixing scheme to accelerate the SCF convergence. Apart from these differences, the calculation follows the same pattern as in the Getting Started tutorial and converges after a few iterations:

In [ ]:
!cat stdout_gs.txt | grep -A 11 "[*] SCF CYCLE ITER #   28 [*]"

<div style="border: 1px solid black; padding: 10px; background-color: #ADD8E6">
<span style="color: blue;"> Note that, as the electrons are non-interacting, there is actually no self-consistency needed. Nevertheless, Octopus takes several SCF iterations to converge. This is because it takes more than one SCF iteration for the iterative eigensolver to converge the wave-functions and eigenvalues.
</div>

To improve this we can try having a better initial guess for the wave-functions by turning the LCAO on. You can do this by setting [LCAOStart](https://www.octopus-code.org/documentation//13/variables/scf/lcao/lcaostart) = lcao_states – compare how many iterations and matrix-vector products (in total) are required now. Why? (Hint: what are Hermite polynomials?)

### Exercises

The first thing to do is to look at the static/info file. Look at the eigenvalue and total energy. Compare to your expectation from the analytic solution to this problem!

We can also play a little bit with the input file and add some other features. For example, what about the other eigenstates of this system? To obtain them we can calculate some unoccupied states. This can be done by changing the [CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) to

[CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) = unocc

in the input file. In this mode Octopus will not only give you the occupied states (which contribute to the density) but also the unoccupied ones. Set the number of states with an extra line

[ExtraStates](https://www.octopus-code.org/documentation//13/variables/states/extrastates) = 10

that will calculate 10 empty states. A thing to note here is that **Octopus** will need the density for this calculation. (Actually for non-interacting electrons the density is not needed for anything, but since **Octopus** is designed for interacting electrons it will try to read the density anyways.) So if you have already performed a static calculation ([CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) = gs) it will just use this result.

In [ ]:
%%writefile inp

stdout = 'stdout_unocc.txt'
stderr = 'stderr_unocc.txt'

FromScratch = yes
CalculationMode = unocc

Dimensions = 1
TheoryLevel = independent_particles

Radius = 10
Spacing = 0.1

%Species
  "HO" | species_user_defined | potential_formula | "0.5*x^2" | valence | 2
%

%Coordinates
  "HO" | 0
%

ExtraStates = 10

In [ ]:
!octopus

Eigenvalues:

In [ ]:
run = Run(".")
run.default.scf.eigenvalues()

If we also want to plot, say, the wave-function, at the end of the calculation, we have to tell **Octopus** to give us this wave-function and how it should do this. We just include

 %[Output](https://www.octopus-code.org/documentation//13/variables/output/output) <br>
    wfs <br>
 % <br>
 [OutputFormat](https://www.octopus-code.org/documentation//13/variables/output/outputformat) = axis_x

The first line tells **Octopus** to give out the wave-functions and the second line says it should do so along the x-axis. We can also select the wave-functions we would like as output, for example the first and second, the fourth and the sixth. (If you don’t specify anything **Octopus** will give them all.)

 [OutputWfsNumber](https://www.octopus-code.org/documentation//13/variables/output/outputwfsnumber) = "1-2,4,6"<br>
 **Octopus** will store the wave-functions in the same folder static where the info file is, under a meaningful name. They are stored as pairs of the ‘‘x’'-coordinate and the value of the wave-function at that position ‘‘x’’. One can easily plot them with python.

In [ ]:
%%writefile inp

stdout = 'stdout_unocc.txt'
stderr = 'stderr_unocc.txt'

FromScratch = yes
CalculationMode = unocc

Dimensions = 1
TheoryLevel = independent_particles

Radius = 10
Spacing = 0.1

%Species
  "HO" | species_user_defined | potential_formula | "0.5*x^2" | valence | 2
%

%Coordinates
  "HO" | 0
%

ExtraStates = 10

%Output
  wfs
%
OutputFormat = axis_x

In [ ]:
!octopus

**Octopus** will store the wave-functions in the same folder static where the info file is, under a meaningful name. They are stored as pairs of the ‘‘x’'-coordinate and the value of the wave-function at that position ‘‘x’’. One can easily plot them with gnuplot or a similar program.

In [ ]:
run = Run()
wf = run.default.scf.wf().isel(step=-1)
wavefunctions = [
    wf.sel(st=1),
    wf.sel(st=2),
    wf.sel(st=4),
    wf.sel(st=6),
]

for wf in wavefunctions:
    plt.plot(wf.x, wf.real, label=wf.name)
plt.legend();

It is also possible to extract a couple of other things from **Octopus** like the density or the Kohn-Sham potential.

[Go to *2-particle-in-a-box.ipynb*](2-particle-in-a-box.ipynb)